In [1]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/e0/7a/08f0ea19a0c835e88aad011083d9dda69a9dfa4585c3453b3bd842eb7bed/selenium-4.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/76/51/12d78ec8abcbda51d8f115d98ebd3ee3da9d9d9af00ac69d3097c5b8d51a/trio-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa670

In [2]:
!pip install chromedriver-py

  Obtaining dependency information for chromedriver-py from https://files.pythonhosted.org/packages/be/9f/57cd0fc7382ef62d2a519cea8ca11be9283282fb7ef75c81ce82c1bde02f/chromedriver_py-126.0.6478.62-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/43.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.3 MB 326.8 kB/s eta 0:02:13
   ---------------------------------------- 0.1/43.3 MB 722.1 kB/s eta 0:01:00
   ---------------------------------------- 0.2/43.3 MB 1.1 MB/s eta 0:00:41
   ---------------------------------------- 0.4/43.3 MB 1.6 MB/s eta 0:00:27
    --------------------------------------- 0.8/43.3 MB 2.6 MB/s eta 0:00:17
    --------------------------------------- 1.0/43.3 MB 3.0 MB/s eta 0:00:14
   - -------------------------------------- 1.7/43.3 MB 4.1 MB/s eta 0:00:11
   - -------------------

In [184]:
import selenium
from selenium import webdriver
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import seaborn as sns
import time
import random
from urllib.parse import urlencode
import requests
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import NoSuchElementException, WebDriverException

In [103]:
filename = "data/booksummaries.txt"
lines = open(filename, encoding="utf8").read().splitlines()
len(lines)

16559

In [104]:
titles = []
authors = []
pubDates = []
genres = []
summaries = []
for line in range(0,len(lines)):
    content = lines[line].split('/m/', 1)[1].split('\t', 1)[1]
    fields = content.split('\t')
    title = fields[0]
    titles.append(title)
    author = fields[1]
    authors.append(author)
    pubDate = fields[2]
    if pubDate == '':
        pubDate = np.nan
    pubDates.append(pubDate)
    genreDict = fields[3]
    genre = ''
    summary = fields[-1]
    try:
        genreDict = eval(genreDict)
        for val in genreDict.values():
            genre += val + ' '
    except:
        genre = np.nan
    
    genres.append(genre)
    summary = summary.replace('\'', '')
    summaries.append(summary)

In [105]:
books_df = pd.DataFrame({'Title':titles, 'Author':authors, 'PublishedDate': pubDates,'Genre':genres, 'Summary':summaries})
books_df

,Title,Author,PublishedDate,Genre,Summary
0,Animal Farm,George Orwell,1945-08-17,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...
...,...,...,...,...,...
16554,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,Transfer of Power,Vince Flynn,2000-06-01,Thriller Fiction,The reader first meets Rapp while he is doing...
16556,Decoded,Jay-Z,2010-11-16,Autobiography,The book follows very rough chronological ord...
16557,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [108]:
book_names = books_df['Title'].tolist()

In [109]:
author_names = books_df['Author'].tolist()

In [224]:
num_books = len(book_names)
num_books

16559

In [115]:
s = Service("chromedriver-win64/chromedriver.exe")
opts = Options()
opts.headless= True
browser = webdriver.Chrome(service=s, options=opts)
browser.get("https://www.goodreads.com/ap/signin?language=en_US&openid.assoc_handle=amzn_goodreads_web_na&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in&siteState=eyJyZXR1cm5fdXJsIjoiaHR0cHM6Ly93d3cuZ29vZHJlYWRzLmNvbS8ifQ%3D%3D")

In [116]:
log_email = browser.find_element(By.ID, value='ap_email')
log_pwd = browser.find_element(By.ID, value='ap_password')
log_email.send_keys("archana.gopal@gmail.com")
log_pwd.send_keys("francon77")
log_pwd.submit()

In [129]:
book_id_list1 = []
for index in range(0,4000):
    browser.get("https://www.goodreads.com/search")
    book_name = book_names[index]
    author_full = author_names[index]
    if(author_full == ''):
        book_id_list.append(np.nan)
        continue
    search_field = browser.find_element(By.ID, value='search_query_main')
    search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
    search_field.send_keys(book_name)
    search_button.click()
    page = urlopen(browser.current_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    books = soup.find_all('tr')
    link = np.nan
    for book in books:
        link = ''
        props = book.find_all('a')
        attributes_book = props[0].attrs
        bookname = attributes_book.get('title')
        link = attributes_book.get('href')
        span = props[2].find('span')
        author = span.string
        if ((book_name in bookname) and (author_full in author)):
            book_ids = re.findall('show\/(\d*).',link)
            book_id_list.append((book_name, book_ids[0]))
            break
    if(link == ''):
        book_id_list.append(np.nan)
    time.sleep(7)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search_query_main"]"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DD4622C2+60002]
	(No symbol) [0x00007FF6DD3DCA59]
	(No symbol) [0x00007FF6DD297EDA]
	(No symbol) [0x00007FF6DD2E76E6]
	(No symbol) [0x00007FF6DD2E77AC]
	(No symbol) [0x00007FF6DD32E9D7]
	(No symbol) [0x00007FF6DD30C2CF]
	(No symbol) [0x00007FF6DD32BCC7]
	(No symbol) [0x00007FF6DD30C033]
	(No symbol) [0x00007FF6DD2D9657]
	(No symbol) [0x00007FF6DD2DA251]
	GetHandleVerifier [0x00007FF6DD773E2D+3278285]
	GetHandleVerifier [0x00007FF6DD7C0190+3590448]
	GetHandleVerifier [0x00007FF6DD7B61D0+3549552]
	GetHandleVerifier [0x00007FF6DD511DE6+779654]
	(No symbol) [0x00007FF6DD3E7ACF]
	(No symbol) [0x00007FF6DD3E2EE4]
	(No symbol) [0x0

In [188]:
book_id_list1 = book_id_list[46:len(book_id_list)]
book_id_list1

[('Animal Farm', '170448'),
 ('A Clockwork Orange', '41817486'),
 ('The Plague', '11989'),
 ('An Enquiry Concerning Human Understanding', '130119'),
 ('A Fire Upon the Deep', '77711'),
 ('All Quiet on the Western Front', '355697'),
 ('A Wizard of Earthsea', '13642'),
 ('Anyone Can Whistle', '10574488'),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ('Children of Dune', '44492286'),
 ('Chapterhouse Dune', '141537891'),
 ('Carmilla', '48037'),
 ('The Cider House Rules', '4687'),
 ('Dracula', '17245'),
 ('Don Quixote', '3836'),
 nan,
 ('Dune Messiah', '44492285'),
 ('Death of a Hero', '15808753'),
 nan,
 ('The Trial', '17690'),
 nan,
 ('Fahrenheit 451', '13079982'),
 ('Gaudy Night', '93575'),
 ('Gylfaginning', '2945155'),
 ('God Emperor of Dune', '44439415'),
 nan,
 ('Gone with the Wind', '18405'),
 ('Heart of Darkness', '4900'),
 ('Hamlet', '1420'),
 ('Heretics of Dune', '44492287'),
 ('Adventures of Huckleberry Finn', '2956'),
 ('Ivanhoe', '6440'),
 ('Johnny Got His Gun',

In [139]:
book_id_list2 = []
for index in range(179,4000):
    browser.get("https://www.goodreads.com/search")
    book_name = book_names[index]
    author_full = author_names[index]
    if(author_full == ''):
        book_id_list2.append(np.nan)
        continue
    search_field = browser.find_element(By.ID, value='search_query_main')
    search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
    search_field.send_keys(book_name)
    search_button.click()
    page = urlopen(browser.current_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    books = soup.find_all('tr')
    link = np.nan
    for book in books:
        link = ''
        props = book.find_all('a')
        attributes_book = props[0].attrs
        bookname = attributes_book.get('title')
        link = attributes_book.get('href')
        span = props[2].find('span')
        author = span.string
        if ((book_name in bookname) and (author_full in author)):
            book_ids = re.findall('show\/(\d*).',link)
            book_id_list2.append((book_name, book_ids[0]))
            break
    if(link == ''):
        book_id_list2.append(np.nan)
    time.sleep(random.randint(2,9))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search_query_main"]"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DD4622C2+60002]
	(No symbol) [0x00007FF6DD3DCA59]
	(No symbol) [0x00007FF6DD297EDA]
	(No symbol) [0x00007FF6DD2E76E6]
	(No symbol) [0x00007FF6DD2E77AC]
	(No symbol) [0x00007FF6DD32E9D7]
	(No symbol) [0x00007FF6DD30C2CF]
	(No symbol) [0x00007FF6DD32BCC7]
	(No symbol) [0x00007FF6DD30C033]
	(No symbol) [0x00007FF6DD2D9657]
	(No symbol) [0x00007FF6DD2DA251]
	GetHandleVerifier [0x00007FF6DD773E2D+3278285]
	GetHandleVerifier [0x00007FF6DD7C0190+3590448]
	GetHandleVerifier [0x00007FF6DD7B61D0+3549552]
	GetHandleVerifier [0x00007FF6DD511DE6+779654]
	(No symbol) [0x00007FF6DD3E7ACF]
	(No symbol) [0x00007FF6DD3E2EE4]
	(No symbol) [0x0

In [142]:
book_id_list2

[("Methuselah's Children", '175319'),
 ("Life: A User's Manual", '28293'),
 ('The Wonderful Wizard of Oz', '236093'),
 ("King Ottokar's Sceptre", '146108'),
 ('Wuthering Heights', '6185'),
 ('Jane Eyre', '10210'),
 ('Catch-22', '168668'),
 ('Robinson Crusoe', '2932'),
 ('King Lear', '12938'),
 ('Clear and Present Danger', '32669'),
 ('Castle Rackrent', '251290'),
 ('Julius Caesar', '13006'),
 ('Emma', '6969'),
 ('Charlie and the Chocolate Factory', '6310'),
 ('The Wasp Factory', '567678'),
 ('Espedair Street', '68772'),
 ('The Crow Road', '12021'),
 ('Consider Phlebas', '11561212'),
 ("Ender's Game", '10644801'),
 ('Titus Andronicus', '72978'),
 ('Measure for Measure', '91571'),
 ('Flow My Tears, The Policeman Said', '61173454'),
 ('Job: A Comedy of Justice', '355'),
 ('Farmer in the Sky', '50851'),
 ('Friday', '17208'),
 ('Red Planet', '50843'),
 ('The Number of the Beast', '50877'),
 ("Foundation's Edge", '76683'),
 ('The Caves of Steel', '41811'),
 ('Prelude to Foundation', '30013')

In [143]:
book_id_list3 = []
for index in range(320,4000):
    browser.get("https://www.goodreads.com/search")
    book_name = book_names[index]
    author_full = author_names[index]
    if(author_full == ''):
        book_id_list3.append(np.nan)
        continue
    search_field = browser.find_element(By.ID, value='search_query_main')
    search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
    search_field.send_keys(book_name)
    search_button.click()
    page = urlopen(browser.current_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    books = soup.find_all('tr')
    link = np.nan
    for book in books:
        link = ''
        props = book.find_all('a')
        attributes_book = props[0].attrs
        bookname = attributes_book.get('title')
        link = attributes_book.get('href')
        span = props[2].find('span')
        author = span.string
        if ((book_name in bookname) and (author_full in author)):
            book_ids = re.findall('show\/(\d*).',link)
            book_id_list3.append((book_name, book_ids[0]))
            break
    if(link == ''):
        book_id_list3.append(np.nan)
    time.sleep(random.randint(4,9))

HTTPError: HTTP Error 504: Gateway Time-out

In [145]:
book_id_list3

[('The Great Gatsby', '4671'),
 ('For Whom the Bell Tolls', '46170'),
 nan,
 ('From Russia with Love', '885849'),
 ('The Tombs of Atuan', '13662'),
 ('Good Omens', '42285661'),
 ('Coraline', '17061'),
 ('The Sandman: Dream Country', '214052476'),
 nan,
 ('Interview with the Vampire', '43763'),
 ('Seven Against Thebes', '752713'),
 nan,
 ('SS-GB', '510539'),
 ('Finnegans Wake', '11013'),
 ('The Virgin Suicides', '10956'),
 ('What Makes Sammy Run?', '101191'),
 ('Born Yesterday', '1105838'),
 ("King Solomon's Mines", '23814'),
 ('Babel-17', '1199688'),
 ('The Languages of Pao', '40889'),
 ("Cat's Cradle", '9605'),
 ('Excellent Women', '178565'),
 ('Becket', '106197'),
 ('The Closing of the American Mind', '75812'),
 ('The Legend of Sleepy Hollow', '721012'),
 ("The Winter's Tale", '44133'),
 ('Dead Air', '12006'),
 nan,
 ('The Exorcist', '179780'),
 ('The Sweet Hereafter', '26924'),
 ('A Severed Head', '11253'),
 ('Coming Up for Air', '8895934'),
 ('Shadow Puppets', '234724'),
 ('The Roa

In [150]:
book_id_list4 = []
for index in range(527,727):
    browser.get("https://www.goodreads.com/search")
    book_name = book_names[index]
    author_full = author_names[index]
    if(author_full == ''):
        book_id_list4.append(np.nan)
        continue
    search_field = browser.find_element(By.ID, value='search_query_main')
    search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
    search_field.send_keys(book_name)
    search_button.click()
    page = urlopen(browser.current_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    books = soup.find_all('tr')
    link = np.nan
    for book in books:
        link = ''
        props = book.find_all('a')
        attributes_book = props[0].attrs
        bookname = attributes_book.get('title')
        link = attributes_book.get('href')
        span = props[2].find('span')
        author = span.string
        if ((book_name in bookname) and (author_full in author)):
            book_ids = re.findall('show\/(\d*).',link)
            book_id_list4.append((book_name, book_ids[0]))
            break
    if(link == ''):
        book_id_list4.append(np.nan)
    time.sleep(random.randint(4,9))

In [152]:
book_id_list4

[('The Tortilla Curtain', '24731'),
 ('Love in a Cold Climate', '50725'),
 ('The Fountainhead', '2122'),
 ('The Last Continent', '47994'),
 ('Laura Blundy', '1079484'),
 ('Contact', '53000876'),
 ('American Gods', '30165203'),
 ('Stardust', '16793'),
 ('Children of the Mind', '31360'),
 ('Wonders of the Invisible World', '814419'),
 ('The Silver Chalice', '122771'),
 ('Thus Spoke Zarathustra', '51893'),
 ('Slaughterhouse-Five', '4981'),
 ('The Name of the Rose', '119073'),
 ('The Member of the Wedding', '330244'),
 ('Magnificent Obsession', '545936'),
 ('Shogun: A Novel of Japan', '56459135'),
 ('Goodnight Moon', '32929'),
 ('The Adventures of Captain Underpants', '207266'),
 ('The Rowan', '61938'),
 ('Lysistrata', '1591'),
 nan,
 ('Ecotopia', '550165'),
 ('The Bad Seed', '467032'),
 ('Life, the Universe and Everything', '8694'),
 ('The Adventures of Tom Sawyer', '24583'),
 ("The Emperor's New Clothes", '343002'),
 ('The Library of Babel', '172366'),
 ('Pebble in the Sky', '41820'),
 (

In [161]:
book_id_list5 = []
for index in range(727,1000):
    browser.get("https://www.goodreads.com/search")
    book_name = book_names[index]
    author_full = author_names[index]
    if(author_full == ''):
        book_id_list5.append(np.nan)
        continue
    search_field = browser.find_element(By.ID, value='search_query_main')
    search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
    search_field.send_keys(book_name)
    search_button.click()
    page = urlopen(browser.current_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    books = soup.find_all('tr')
    link = np.nan
    for book in books:
        link = ''
        props = book.find_all('a')
        attributes_book = props[0].attrs
        bookname = attributes_book.get('title')
        link = attributes_book.get('href')
        span = props[2].find('span')
        author = span.string
        if ((book_name in bookname) and (author_full in author)):
            book_ids = re.findall('show\/(\d*).',link)
            book_id_list5.append((book_name, book_ids[0]))
            break
    if(link == ''):
        book_id_list5.append(np.nan)
    time.sleep(random.randint(4,9))

In [163]:
book_id_list5

[('Trumps of Doom', '536790'),
 ('Dream of the Red Chamber', '535739'),
 ('Nice Work', '69935'),
 ('The Little Engine That Could', '824204'),
 ('Naked Lunch', '7437'),
 ('Pattern Recognition', '22320'),
 nan,
 ('The Chrysalids', '826845'),
 ('Who Goes There?', '6468870'),
 ('Blood of Amber', '239924'),
 ('Sign of Chaos', '62004'),
 ('Porno', '23965'),
 ('Prince of Chaos', '61994'),
 ('The Love of the Last Tycoon', '16857'),
 ('Save Me the Waltz', '150104'),
 ('It', '830502'),
 ('The Amazing Maurice and His Educated Rodents', '34534'),
 ('Permutation City', '156784'),
 nan,
 ('The Wee Free Men', '34494'),
 ('Strata', '34493'),
 ('The Dark Side of the Sun', '181010'),
 ('Soul Music', '34502'),
 ('Bluebeard', '9601'),
 nan,
 ('Thunderball', '177194'),
 ("The Devil's Disciple", '255019'),
 ('The Scarlet Letter', '12296'),
 ('The G-String Murders', '150065985'),
 ('Live from Death Row', '449916'),
 ('The Blue Lotus', '77174'),
 ('Mrs Craddock', '879162'),
 ('Hideous Kinky', '283786'),
 ('My

In [172]:
book_id_list6 = []
for index in range(1000,5000):
    try:
        browser.get("https://www.goodreads.com/search")
        book_name = book_names[index]
        author_full = author_names[index]
        if(author_full == ''):
            book_id_list6.append(np.nan)
            continue
        search_field = browser.find_element(By.ID, value='search_query_main')
        search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
        search_field.send_keys(book_name)
        search_button.click()
        page = urlopen(browser.current_url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        books = soup.find_all('tr')
        link = np.nan
        for book in books:
            link = ''
            props = book.find_all('a')
            attributes_book = props[0].attrs
            bookname = attributes_book.get('title')
            link = attributes_book.get('href')
            span = props[2].find('span')
            author = span.string
            if ((book_name in bookname) and (author_full in author)):
                book_ids = re.findall('show\/(\d*).',link)
                book_id_list6.append((book_name, book_ids[0]))
                break
        if(link == ''):
            book_id_list6.append(np.nan)
        time.sleep(random.randint(4,9))
    except HTTPError:
        time.sleep(2)
        book_id_list6.append(np.nan)
    except NoSuchElementException:
        time.sleep(2)
        book_id_list6.append(np.nan)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF6DD4622C2+60002]
	(No symbol) [0x00007FF6DD3DCA59]
	(No symbol) [0x00007FF6DD297EDA]
	(No symbol) [0x00007FF6DD28FEC2]
	(No symbol) [0x00007FF6DD2809E4]
	(No symbol) [0x00007FF6DD282232]
	(No symbol) [0x00007FF6DD280D11]
	(No symbol) [0x00007FF6DD2804F1]
	(No symbol) [0x00007FF6DD280430]
	(No symbol) [0x00007FF6DD27E3BD]
	(No symbol) [0x00007FF6DD27E9FC]
	(No symbol) [0x00007FF6DD29ACB1]
	(No symbol) [0x00007FF6DD32C66E]
	(No symbol) [0x00007FF6DD30C28A]
	(No symbol) [0x00007FF6DD32BCC7]
	(No symbol) [0x00007FF6DD30C033]
	(No symbol) [0x00007FF6DD2D9657]
	(No symbol) [0x00007FF6DD2DA251]
	GetHandleVerifier [0x00007FF6DD773E2D+3278285]
	GetHandleVerifier [0x00007FF6DD7C0190+3590448]
	GetHandleVerifier [0x00007FF6DD7B61D0+3549552]
	GetHandleVerifier [0x00007FF6DD511DE6+779654]
	(No symbol) [0x00007FF6DD3E7ACF]
	(No symbol) [0x00007FF6DD3E

In [178]:
print(book_id_list6)

[('Forever Peace', '21618'), ('Alaska', '42953'), ('The Shooting Star', '146107'), ('The Mill on the Floss', '20564'), ('Lullaby', '22206'), ('Le Père Goriot', '159977'), ('Lucky Jim', '395182'), ("A Connecticut Yankee in King Arthur's Court", '162898'), ('The Glass Menagerie', '92517'), ('Absolution Gap', '89184'), ('A Delicate Balance', '135503'), ('Wolves of the Calla', '4978'), ('The Good Earth', '1078'), ('The Secret Garden', '2998'), ('The Sea, the Sea', '11229'), ('Trinity', '42696'), ('The Discovery of Heaven', '88389'), ('A Prayer for Owen Meany', '4473'), ('This Present Darkness', '17309'), nan, ('Get Shorty', '702539'), ('Autobiography of a Brown Buffalo', '197451'), ('The Naked Sun', '30016'), ('Farewell, My Lovely', '2050'), ('The Long Goodbye', '2054'), ('Relic', '67035'), ('Life of Pi', '4214'), ("Le Morte d'Arthur", '672875'), ('The Robots of Dawn', '41810'), ('Peril at End House', '16424'), ('The Subtle Knife', '41637836'), ('The Story of the Stone', '77207'), ('The Mi

In [181]:
book_id_list7 = []
for index in range(2317,5000):
    try:
        browser.get("https://www.goodreads.com/search")
        book_name = book_names[index]
        author_full = author_names[index]
        if(author_full == ''):
            book_id_list7.append(np.nan)
            continue
        search_field = browser.find_element(By.ID, value='search_query_main')
        search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
        search_field.send_keys(book_name)
        search_button.click()
        page = urlopen(browser.current_url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        books = soup.find_all('tr')
        link = np.nan
        for book in books:
            link = ''
            props = book.find_all('a')
            attributes_book = props[0].attrs
            bookname = attributes_book.get('title')
            link = attributes_book.get('href')
            span = props[2].find('span')
            author = span.string
            if ((book_name in bookname) and (author_full in author)):
                book_ids = re.findall('show\/(\d*).',link)
                book_id_list7.append((book_name, book_ids[0]))
                break
        if(link == ''):
            book_id_list7.append(np.nan)
        time.sleep(random.randint(4,9))
    except HTTPError:
        time.sleep(2)
        book_id_list7.append(np.nan)
    except NoSuchElementException:
        time.sleep(2)
        book_id_list7.append(np.nan)
    except WebDriverException:
        time.sleep(5)
        book_id_list7.append(np.nan)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [182]:
len(book_id_list7)

1353

In [183]:
print(book_id_list7)

[nan, ('No Deals, Mr. Bond', '143573740'), ('Voyage of the Damned', '619930'), ('Brokenclaw', '164765674'), ('Never Send Flowers', '207248967'), ('The Facts of Death', '432507'), ('High Time to Kill', '32547'), ('Doubleshot', '432515'), ('Never Dream of Dying', '432513'), ('The Man with the Red Tattoo', '7451352'), ('Facundo', '269735'), ('The Ambushers', '1085487'), ('Crusade in Jeans', '974324'), ("Smilla's Sense of Snow", '124509'), ('A View from the Bridge', '847168'), nan, ('The Chimes', '169848'), ('The Cricket on the Hearth', '169848'), ('Phaedo', '30292'), ('Michael Strogoff', '10970257'), ('Thérèse Raquin', '110871'), nan, ('Kaleidoscope Century', '1365524'), ('Bonjour Tristesse', '4502660'), ('Return to Peyton Place', '226448'), ('Eastern Standard Tribe', '29589'), ('The Gold Bug Variations', '23012'), ('The Golem', '99794'), ('Storm of Steel', '240485'), ('Timeline', '7669'), ('Time in Advance', '1836359'), ('Danse Macabre', '11563'), ('Practical Demonkeeping', '33457'), ('W

In [203]:
#Process all lists so far and enter into dataframe
import math
main_list_1 = book_id_list1 + book_id_list2 + book_id_list3 + book_id_list4 + book_id_list5 + book_id_list6 + book_id_list7
books_data_set1 = books_df.iloc[0:4048,:]
books_data_set1.tail()
books_data_set1['BookID'] = np.nan
for val_tuple in main_list_1:
    if(isinstance(val_tuple, tuple)):
        name = val_tuple[0]
        book_id = val_tuple[1]
        books_data_set1['BookID'] = np.where(name == books_data_set1['Title'], book_id,   books_data_set1['BookID'])

C:\temp\ipykernel_24812\396184766.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set1['BookID'] = np.nan
C:\temp\ipykernel_24812\396184766.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set1['BookID'] = np.where(name == books_data_set1['Title'], book_id,   books_data_set1['BookID'])


In [214]:
books_data_set1['BookID'] = books_data_set1['BookID'].astype(str)
books_data_set1['BookID'] = np.where(books_data_set1['BookID'] == 'nan', np.nan, books_data_set1['BookID'])
books_data_set1['BookID'].isna().sum()
books_data_set1 = books_data_set1[['Title', 'BookID', 'Author', 'PublishedDate', 'Genre', 'Summary']]
books_data_set1

,Title,BookID,Author,PublishedDate,Genre,Summary
0,Animal Farm,170448,George Orwell,1945-08-17,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,41817486,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,11989,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,130119,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,77711,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...
...,...,...,...,...,...,...
4043,Brilliance of the Moon,NaN,Gillian Rubinstein,2004,Speculative fiction Fantasy,"Soon after Kaede and Takeos marriage, messeng..."
4044,Dragoncharm,61341767,Graham Edwards,1995-02,Speculative fiction Fantasy Fiction,The story begins with the destruction of the ...
4045,Deathstalker,NaN,Simon Green,NaN,Science Fiction Speculative fiction,- --&#62; &#60;!-
4046,Junk,98973,Melvin Burgess,1996-11-14,Literary realism Children's literature Young a...,The novel is told in the first person from va...


In [215]:
books_data_set1.to_csv('books_data_set1.csv')

In [212]:
book_id_list8 = []
for index in range(4048,10000):
    try:
        browser.get("https://www.goodreads.com/search")
        book_name = book_names[index]
        author_full = author_names[index]
        if(author_full == ''):
            book_id_list8.append(np.nan)
            continue
        search_field = browser.find_element(By.ID, value='search_query_main')
        search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
        search_field.send_keys(book_name)
        search_button.click()
        page = urlopen(browser.current_url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        books = soup.find_all('tr')
        link = np.nan
        for book in books:
            link = ''
            props = book.find_all('a')
            attributes_book = props[0].attrs
            bookname = attributes_book.get('title')
            link = attributes_book.get('href')
            span = props[2].find('span')
            author = span.string
            if ((book_name in bookname) and (author_full in author)):
                book_ids = re.findall('show\/(\d*).',link)
                book_id_list8.append((book_name, book_ids[0]))
                break
        if(link == ''):
            book_id_list8.append(np.nan)
        time.sleep(random.randint(4,9))
    except HTTPError:
        time.sleep(2)
        book_id_list8.append(np.nan)
    except NoSuchElementException:
        time.sleep(2)
        book_id_list8.append(np.nan)
    except WebDriverException:
        time.sleep(5)
        book_id_list8.append(np.nan)
    except URLError:
        time.sleep(3)
        book_id_list8.append(np.nan)

In [213]:
len(book_id_list8)

4437

In [217]:
print(book_id_list8)

[('The Dream Master', '61992'), nan, ('This Happy Breed', '8777384'), ('Cast of Shadows', '30195'), ("Beard's Roman Women", '230117'), nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ('The Mystery of the Fire Dragon', '862005'), nan, ('The Simulacra', '226460'), ('HMS Ulysses', '149667'), ('Tunnel Through the Deeps', '1728828'), ('Herr Lehmann', '4485'), ('In the Wet', '181980'), ("Zeno's Conscience", '84737'), ('Indian Ink', '17905'), ('The Homeward Bounders', '47570'), ('A Darkling Plain', '219110'), ('The Soddit', '1066659'), ('The Stone Angel', '1102322'), ('The Incredible and Sad Tale of Innocent Eréndira and her Heartless Grandmother', '200839009'), ('The Parsifal Mosaic', '251823'), ('Nor Crystal Tears', '35134'), ('The Mysteries of Harris Burdick', '55734'), ('The Forest of Doom', '31172003'), ('City of Thieves', '1261007'), ("A Madman's Diary", '21442812'), ('The 48 Laws of Power', '1303'), ('The Eight Doctors', '71364'), ('Double Fudge', '80672'), ('Q-in

In [218]:
main_list_2 = book_id_list8
books_data_set2 = books_df.iloc[4048:10000,:]
books_data_set2.tail()
books_data_set2['BookID'] = np.nan
for val_tuple in main_list_2:
    if(isinstance(val_tuple, tuple)):
        name = val_tuple[0]
        book_id = val_tuple[1]
        books_data_set2['BookID'] = np.where(name == books_data_set2['Title'], book_id,   books_data_set2['BookID'])

C:\temp\ipykernel_24812\4058645591.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set2['BookID'] = np.nan
C:\temp\ipykernel_24812\4058645591.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set2['BookID'] = np.where(name == books_data_set2['Title'], book_id,   books_data_set2['BookID'])


In [220]:
books_data_set2.tail()

,Title,Author,PublishedDate,Genre,Summary,BookID
9995,The Lost Warrior,Erin Hunter,2007-04-24,Children's literature Fantasy,The Lost Warrior opens with narration from Gr...,710187
9996,A Necessary End,Peter Robinson,1989,Crime Fiction Mystery,Violence erupts at an anti-nuclear demonstrat...,102168
9997,The Hanging Valley,Peter Robinson,1989,Crime Fiction Mystery,"A faceless, maggot-ridden corpse is discovere...",102175
9998,Past Reason Hated,Peter Robinson,1991,Crime Fiction Mystery,The body of Caroline Hartley is found one eve...,102184
9999,Wednesday's Child,Peter Robinson,1992,Crime Fiction Mystery,"When a well-dressed couple, claiming to be so...",102188


In [221]:
books_data_set2['BookID'] = books_data_set2['BookID'].astype(str)
books_data_set2['BookID'] = np.where(books_data_set2['BookID'] == 'nan', np.nan, books_data_set2['BookID'])
books_data_set2['BookID'].isna().sum()
books_data_set2 = books_data_set2[['Title', 'BookID', 'Author', 'PublishedDate', 'Genre', 'Summary']]
books_data_set2

C:\temp\ipykernel_24812\647103363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set2['BookID'] = books_data_set2['BookID'].astype(str)
C:\temp\ipykernel_24812\647103363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_set2['BookID'] = np.where(books_data_set2['BookID'] == 'nan', np.nan, books_data_set2['BookID'])


,Title,BookID,Author,PublishedDate,Genre,Summary
4048,Millennium People,NaN,J. G. Ballard,NaN,Speculative fiction,When a bomb explodes on a baggage carousel at...
4049,The Dream Master,61992,Roger Zelazny,1966,Science Fiction,The Dream Master is set in a future where the...
4050,Spiral,NaN,Koji Suzuki,1995,Horror,The events in the story occur a day after the...
4051,Life Against Death: The Psychoanalytic Meaning...,NaN,,1959,Psychology Literary theory,Brown offers what Robinson sees as a unique a...
4052,This Happy Breed,8777384,Noël Coward,NaN,NaN,;Scene 1 – June 1919 The Gibbons family has j...
...,...,...,...,...,...,...
9995,The Lost Warrior,710187,Erin Hunter,2007-04-24,Children's literature Fantasy,The Lost Warrior opens with narration from Gr...
9996,A Necessary End,102168,Peter Robinson,1989,Crime Fiction Mystery,Violence erupts at an anti-nuclear demonstrat...
9997,The Hanging Valley,102175,Peter Robinson,1989,Crime Fiction Mystery,"A faceless, maggot-ridden corpse is discovere..."
9998,Past Reason Hated,102184,Peter Robinson,1991,Crime Fiction Mystery,The body of Caroline Hartley is found one eve...


In [222]:
books_data_set2.to_csv('books_data_set2.csv')

In [228]:
book_id_list9 = []
for index in range(10000,num_books):
    success = False
    while(not success):
        try:
            browser.get("https://www.goodreads.com/search")
            book_name = book_names[index]
            author_full = author_names[index]
            if(author_full == ''):
                book_id_list9.append(np.nan)
                success = True
                continue
            search_field = browser.find_element(By.ID, value='search_query_main')
            search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
            search_field.send_keys(book_name)
            search_button.click()
            page = urlopen(browser.current_url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            books = soup.find_all('tr')
            link = np.nan
            for book in books:
                link = ''
                props = book.find_all('a')
                attributes_book = props[0].attrs
                bookname = attributes_book.get('title')
                link = attributes_book.get('href')
                span = props[2].find('span')
                author = span.string
                if ((book_name in bookname) and (author_full in author)):
                    book_ids = re.findall('show\/(\d*).',link)
                    book_id_list9.append((book_name, book_ids[0]))
                    break
            if(link == ''):
                book_id_list9.append(np.nan)
            time.sleep(random.randint(4,9))
            success=True
        except HTTPError:
            time.sleep(3)
            success = False
        except NoSuchElementException:
            time.sleep(3)
            success = False
        except WebDriverException:
            time.sleep(5)
            success = False
        except URLError:
            time.sleep(3)
            success = False

In [229]:
len(book_id_list9)

4994

In [231]:
print(book_id_list9)

[('Dry Bones that Dream', '377913'), ('Innocent Graves', '102206'), ('Dead Right', '11861546'), ('In a Dry Season', '9082'), ('The Silent and the Damned', '19478791'), ('The Hidden Assassins', '62545'), ('Grimble', '80702'), ('Cold is the Grave', '77514'), ('The Summer That Never Was', '11177331'), ('Strange Affair', '102268'), ('Piece of My Heart: A Novel of Suspense', '134822741'), ('Friend of the Devil', '989584'), ('The Almost Moon', '783291'), ('Playing for Pizza', '1205297'), nan, ("The Devil's Law Case", '130594568'), ('Between Mom and Jo', '496102'), ('The Savage Detectives', '63033'), ('Maria: or, The Wrongs of Woman', '1656262'), nan, ('The Bourne Ultimatum', '31259'), nan, ('The Execution Channel', '185829'), nan, ('The Proteus Operation', '849493'), ('Mélusine', '492069'), ('The Virtu', '219811'), ('Just Listen', '51738'), ('Evening Class', '41967'), ('Quentins', '41950'), ('Love and Other Impossible Pursuits', '165145'), nan, ('Children of God', '16948'), ('Swordbird', '11

In [232]:
main_list = main_list_1 + book_id_list8 + book_id_list9
books_data_set = books_df.copy()
books_data_set['BookID'] = np.nan
for val_tuple in main_list:
    if(isinstance(val_tuple, tuple)):
        name = val_tuple[0]
        book_id = val_tuple[1]
        books_data_set['BookID'] = np.where(name == books_data_set['Title'], book_id,   books_data_set['BookID'])

In [233]:
books_data_set

,Title,Author,PublishedDate,Genre,Summary,BookID
0,Animal Farm,George Orwell,1945-08-17,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca...",170448
1,A Clockwork Orange,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan...",41817486
2,The Plague,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...,11989
3,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...,130119
4,A Fire Upon the Deep,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...,77711
...,...,...,...,...,...,...
16554,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ...",10353049
16555,Transfer of Power,Vince Flynn,2000-06-01,Thriller Fiction,The reader first meets Rapp while he is doing...,184655
16556,Decoded,Jay-Z,2010-11-16,Autobiography,The book follows very rough chronological ord...,8425806
16557,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...,nan


In [234]:
books_data_set = books_data_set[['Title', 'BookID', 'Author', 'PublishedDate', 'Genre', 'Summary']]
books_data_set

,Title,BookID,Author,PublishedDate,Genre,Summary
0,Animal Farm,170448,George Orwell,1945-08-17,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,41817486,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,11989,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,130119,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,77711,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...
...,...,...,...,...,...,...
16554,Under Wildwood,10353049,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,Transfer of Power,184655,Vince Flynn,2000-06-01,Thriller Fiction,The reader first meets Rapp while he is doing...
16556,Decoded,8425806,Jay-Z,2010-11-16,Autobiography,The book follows very rough chronological ord...
16557,America Again: Re-becoming The Greatness We Ne...,nan,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [235]:
books_data_set.to_csv('books_data_set.csv')

In [243]:
books_df_clean = pd.read_csv('books_data_set_to_clean.csv', low_memory=False, dtype={"Title": "string", "BookID": object, "Author": object, "PublishedDate": object,
                                                                  "Genre": object, "Summary": object})
books_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16522 entries, 0 to 16521
Columns: 163 entries, Unnamed: 0 to Unnamed: 162
dtypes: object(162), string(1)
memory usage: 20.5+ MB


In [244]:
books_df_clean

,Unnamed: 0,Title,BookID,Author,PublishedDate,Genre,Summary,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162
0,0,Animal Farm,170448,George Orwell,8/17/1945,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A Clockwork Orange,41817486,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,The Plague,11989,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,An Enquiry Concerning Human Understanding,130119,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,A Fire Upon the Deep,77711,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16517,16554,Under Wildwood,10353049,Colin Meloy,9/25/2012,NaN,"Prue McKeel, having rescued her brother from ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16518,16555,Transfer of Power,184655,Vince Flynn,6/1/2000,Thriller Fiction,The reader first meets Rapp while he is doing...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16519,16556,Decoded,8425806,Jay-Z,11/16/2010,Autobiography,The book follows very rough chronological ord...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16520,16557,America Again: Re-becoming The Greatness We Ne...,NaN,Stephen Colbert,10/2/2012,NaN,Colbert addresses topics including Wall Stree...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
col_to_remove = []
for col in books_df_clean.columns:
    if("Unnamed" in col):
        col_to_remove.append(col)
    
books_df_clean = books_df_clean.drop(columns=col_to_remove, axis=1)
books_df_clean

,Title,BookID,Author,PublishedDate,Genre,Summary
0,Animal Farm,170448,George Orwell,8/17/1945,Roman à clef Satire Children's literature Spec...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,41817486,Anthony Burgess,1962,Science Fiction Novella Speculative fiction Ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,11989,Albert Camus,1947,Existentialism Fiction Absurdist fiction Novel,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,130119,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,77711,Vernor Vinge,NaN,Hard science fiction Science Fiction Speculati...,The novel posits that space around the Milky ...
...,...,...,...,...,...,...
16517,Under Wildwood,10353049,Colin Meloy,9/25/2012,NaN,"Prue McKeel, having rescued her brother from ..."
16518,Transfer of Power,184655,Vince Flynn,6/1/2000,Thriller Fiction,The reader first meets Rapp while he is doing...
16519,Decoded,8425806,Jay-Z,11/16/2010,Autobiography,The book follows very rough chronological ord...
16520,America Again: Re-becoming The Greatness We Ne...,NaN,Stephen Colbert,10/2/2012,NaN,Colbert addresses topics including Wall Stree...


In [252]:
books_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16522 entries, 0 to 16521
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          16522 non-null  string
 1   BookID         10240 non-null  object
 2   Author         14204 non-null  object
 3   PublishedDate  10944 non-null  object
 4   Genre          12831 non-null  object
 5   Summary        16522 non-null  object
dtypes: object(5), string(1)
memory usage: 774.6+ KB


In [271]:
books_df_revised = books_df_clean.dropna(subset=['Author'])
books_df_revised.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14204 entries, 0 to 16521
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          14204 non-null  string
 1   BookID         10240 non-null  object
 2   Author         14204 non-null  object
 3   PublishedDate  10306 non-null  object
 4   Genre          12056 non-null  object
 5   Summary        14204 non-null  object
dtypes: object(5), string(1)
memory usage: 776.8+ KB


In [257]:
books_to_check = books_df_revised[books_df_revised['BookID'].isna()][['Title', 'Author']]
books_to_check.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3964 entries, 8 to 16521
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   3964 non-null   string
 1   Author  3964 non-null   object
dtypes: object(1), string(1)
memory usage: 92.9+ KB


In [258]:
book_titles = books_to_check['Title'].tolist()
book_titles
book_authors = books_to_check['Author'].tolist()

In [259]:
book_id_list10 = []
for index in range(0, len(book_titles)):
    success = False
    while(not success):
        try:
            browser.get("https://www.goodreads.com/search")
            book_name = book_titles[index]
            author_full = book_authors[index]
            search_field = browser.find_element(By.ID, value='search_query_main')
            search_button = browser.find_element(By.CLASS_NAME, value='searchBox__button')
            search_field.send_keys(book_name)
            search_button.click()
            page = urlopen(browser.current_url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            books = soup.find_all('tr')
            link = ''
            for book in books:
                link = ''
                props = book.find_all('a')
                attributes_book = props[0].attrs
                bookname = attributes_book.get('title')
                link = attributes_book.get('href')
                span = props[2].find('span')
                author = span.string
                if ((book_name in bookname) and (author_full in author)):
                    book_ids = re.findall('show\/(\d*).',link)
                    book_id_list10.append((book_name, book_ids[0]))
                    break
            if(link == ''):
                book_id_list10.append(np.nan)
            time.sleep(random.randint(4,9))
            success=True
        except HTTPError:
            time.sleep(3)
            success = False
        except NoSuchElementException:
            time.sleep(3)
            success = False
        except WebDriverException:
            time.sleep(5)
            success = False
        except URLError:
            time.sleep(3)
            success = False

In [260]:
len(book_id_list10)

217

In [262]:
print(book_id_list10)

[('Smith of Wootton Major', '210434827'), ('The Hustler', '323170'), ('The Sandman: Season of Mists', '169028877'), ('Tartarin de Tarascon', '44291207'), nan, ('The Courts of Chaos', '359581'), ('Come Back, Little Sheba', '134031'), ('Hay Fever', '370966'), ('The Bridge of San Luis Rey', '92508'), ('Exercitatio Anatomica de Motu Cordis et Sanguinis in Animalibus', '161687051'), ('Free Culture: How Big Media Uses Technology and the Law to Lock Down Culture and Control Creativity', '72016'), ('Death on the Nile', '131359'), ('Team Yankee', '34570'), ('The Rules of Attraction', '9912'), ('The Changeling', '351605'), ('Flicker', '243162'), nan, ('Misspent Youth', '45248'), ('Asterix and the Great Divide', '58724844'), ('The Ragwitch', '47621'), nan, ('Queen of Angels', '410766'), ('The Happy Return', '207911012'), ('Sandman: The Dream Hunters', '166580'), ('Wild Swans', '1848'), ('Mary Poppins Opens the Door', '141725404'), ('West of Eden', '325661'), ('The City of Ember', '307791'), ('The

In [272]:

for val_tuple in book_id_list10:
    if(isinstance(val_tuple, tuple)):
        name = val_tuple[0]
        book_id = val_tuple[1]
        books_df_revised['BookID'] = np.where(name == books_df_revised['Title'], book_id,   books_df_revised['BookID'])

C:\temp\ipykernel_24812\245037269.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df_revised['BookID'] = np.where(name == books_df_revised['Title'], book_id,   books_df_revised['BookID'])
C:\temp\ipykernel_24812\245037269.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df_revised['BookID'] = np.where(name == books_df_revised['Title'], book_id,   books_df_revised['BookID'])
C:\temp\ipykernel_24812\245037269.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [273]:
books_df_revised.isna().sum()

Title               0
BookID           3812
Author              0
PublishedDate    3898
Genre            2148
Summary             0
dtype: int64

In [274]:
books_df_revised.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14204 entries, 0 to 16521
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          14204 non-null  string
 1   BookID         10392 non-null  object
 2   Author         14204 non-null  object
 3   PublishedDate  10306 non-null  object
 4   Genre          12056 non-null  object
 5   Summary        14204 non-null  object
dtypes: object(5), string(1)
memory usage: 776.8+ KB


In [275]:
books_df_revised = books_df_revised.dropna(subset=['BookID'])

In [276]:
books_df_revised.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10392 entries, 0 to 16519
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          10392 non-null  string
 1   BookID         10392 non-null  object
 2   Author         10392 non-null  object
 3   PublishedDate  7639 non-null   object
 4   Genre          8877 non-null   object
 5   Summary        10392 non-null  object
dtypes: object(5), string(1)
memory usage: 568.3+ KB


In [277]:
books_df_revised.to_csv('final_books_data.csv')